# IPython Blockfolio

This is a simple "Blockfolio" using Python and the [CoinMarketCap API](https://coinmarketcap.com/api/).

It will show the net worth in USD of your Cryptocurrency investments as well as the relative share
of coins you own in comparison to the total coin supply of the corresponding digital asset.

First, the current exchanges rates are loaded, then combined with your investments, and finally displayed using Plotly.

This notebook can be found on [my github profile](https://github.com/SmokinCaterpillar/blockfolio).

Imports
--------

In [1]:
import pandas as pd

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

## Loading Data from CoinMarkteCap

In [2]:
limit = 200  # only load the top 200 currencies, if you have invested in smaller ones increase this limit
coin_market_cap_api = 'https://api.coinmarketcap.com/v1/ticker/?limit={}'.format(limit)
coin_market_cap_api


'https://api.coinmarketcap.com/v1/ticker/?limit=200'

In [3]:
# load the data using pandas and keep track of the time
now = pd.datetime.now()
market_data = pd.read_json(coin_market_cap_api)
market_data.head(10)

,24h_volume_usd,available_supply,id,last_updated,market_cap_usd,max_supply,name,percent_change_1h,percent_change_24h,percent_change_7d,price_btc,price_usd,rank,symbol,total_supply
0,1.294180e+10,16773587,bitcoin,1514726060,223632171319,2.100000e+07,Bitcoin,0.39,1.93,-1.49,1.000000,13332.400000,1,BTC,16773587
1,6.231430e+09,38739144847,ripple,1514726341,91761799791,1.000000e+11,Ripple,2.95,2.94,133.02,0.000180,2.368710,2,XRP,99993093880
2,2.748370e+09,96683112,ethereum,1514726348,70861179492,NaN,Ethereum,0.41,3.49,10.37,0.055800,732.922000,3,ETH,96683112
3,1.276980e+09,16885938,bitcoin-cash,1514726355,41335761844,2.100000e+07,Bitcoin Cash,-0.32,-1.42,-12.72,0.186369,2447.940000,4,BCH,16885938
4,4.170360e+08,25927070538,cardano,1514726357,18553385750,4.500000e+10,Cardano,2.14,15.64,91.19,0.000054,0.715599,5,ADA,31112483745
5,8.302340e+08,54553508,litecoin,1514726341,12067454218,8.400000e+07,Litecoin,0.91,-0.56,-17.37,0.016841,221.204000,6,LTC,54553508
6,2.256400e+08,2779530283,iota,1514726354,9564474885,2.779530e+09,IOTA,-0.13,7.25,2.55,0.000262,3.441040,7,MIOTA,2779530283
7,4.954830e+07,8999999999,nem,1514726344,8627534999,NaN,NEM,-0.54,0.85,6.20,0.000073,0.958615,8,XEM,8999999999
8,1.591500e+08,7786094,dash,1514726341,7920871288,1.890000e+07,Dash,0.46,0.56,-12.25,0.077451,1017.310000,9,DASH,7786094
9,2.657510e+08,17858964417,stellar,1514726343,5931194249,NaN,Stellar,0.62,4.07,55.56,0.000025,0.332113,10,XLM,103550799663


## Your Blockfolio

Below you can fill in the types and amount of coins you have into the Python dictionary
with the format `'SYMBOL': value`.

In [4]:
# enter your coins below:

block_folio = {
    'BTC': 0.31337,
    'BCH': 0.01337,
    'ETH': 3,
    'LTC': 1.5,
    'MIOTA': 777,
    'XRP': 13,
    'DASH': 0.1, 
    'XMR': 1, 
    'LSK': 12, 
    'OMG': 1.1, 
    'XRB': 42, 
    'PIVX': 123, 
    'ARK': 22, 
    'NEO': 7, 
    'GAS': 0.01,
    'STEEM': 499,
}

# display your blockfolio
block_folio = pd.DataFrame(list(block_folio.items()), columns=['symbol', 'amount'])
block_folio.head(len(block_folio))

,symbol,amount
0,DASH,0.10000
1,BTC,0.31337
2,STEEM,499.00000
3,ARK,22.00000
4,PIVX,123.00000
5,XRB,42.00000
6,LSK,12.00000
7,ETH,3.00000
8,LTC,1.50000
9,OMG,1.10000


### Combining the Blockfolio and the API Data

In [5]:
# merge the API and blockfolio data and sort by investment value
merged_data = block_folio.merge(market_data, how='left')
merged_data['value_usd'] = merged_data.amount * merged_data.price_usd
merged_data['coinshare'] = merged_data.amount / merged_data.available_supply
merged_data = merged_data.sort_values('value_usd', ascending=False)
merged_data.head()

,symbol,amount,24h_volume_usd,available_supply,id,last_updated,market_cap_usd,max_supply,name,percent_change_1h,percent_change_24h,percent_change_7d,price_btc,price_usd,rank,total_supply,value_usd,coinshare
1,BTC,0.31337,1.294180e+10,16773587,bitcoin,1514726060,223632171319,2.100000e+07,Bitcoin,0.39,1.93,-1.49,1.000000,13332.40000,1,16773587,4177.974188,1.868235e-08
10,MIOTA,777.00000,2.256400e+08,2779530283,iota,1514726354,9564474885,2.779530e+09,IOTA,-0.13,7.25,2.55,0.000262,3.44104,7,2779530283,2673.688080,2.795436e-07
7,ETH,3.00000,2.748370e+09,96683112,ethereum,1514726348,70861179492,NaN,Ethereum,0.41,3.49,10.37,0.055800,732.92200,3,96683112,2198.766000,3.102920e-08
2,STEEM,499.00000,3.666740e+06,246705222,steem,1514726346,685127538,NaN,Steem,0.31,3.37,-11.05,0.000211,2.77711,40,263679316,1385.777890,2.022657e-06
4,PIVX,123.00000,4.719910e+06,55247274,pivx,1514726346,543487325,NaN,PIVX,1.68,3.29,-16.39,0.000749,9.83736,49,55247274,1209.995280,2.226354e-06


### Your Blockfolio in numbers and graphs!


In [6]:
networth = 'Your blockfolio is currently (i.e {}) worth {:.2f} USD!'.format(now.strftime('%Y-%m-%d %I:%M %p'), 
                                                                        merged_data.value_usd.sum())
print(networth)

Your blockfolio is currently (i.e 2017-12-31 02:18 PM) worth 13984.80 USD!


#### Net Worth

Let's plot the worth of each asset in descending order:

In [7]:
marker = dict(color='rgb(158,202,225)',
              line=dict(color='rgb(8,48,107)', width=1.5))

layout = go.Layout(title='Blockfolio Networth in USD',
                   xaxis=dict(title='Cryptocurrency'),
                   yaxis=dict(title='USD'))
              
value_chart=go.Bar(x=merged_data.symbol, y=merged_data.value_usd, marker=marker)

fig = go.Figure(data=[value_chart], layout=layout)
py.iplot(fig)

#### Fraction of total Supply
Finally, that looks at the relative sizes of your investment. 
This gives you an idea or visualizes in which cryptocurrency you believe in the most. 
The graph below shows you fractional share of the the current crypto's total supply.

In [8]:
marker = dict(color='rgb(258,202,125)', line=dict(color='rgb(8,48,107)', width=1.5,))

layout = go.Layout(title='Relative Blockfolio Size',
                   xaxis=dict(title='Cryptocurrency'),
                   yaxis=dict(title='Fraction of total Supply'))

share_chart=go.Bar(x=merged_data.symbol, y=merged_data.coinshare, marker=marker)

fig = go.Figure(data=[share_chart], layout=layout)
py.iplot(fig)